<div class="alert alert-success" style="font-size: 16px;">
    <div style="font-size: 22px; font-weight:bold; margin: 10px 0px;">Topic Modeling</div>
After preprocessing we have to vectorize our preprocessed text and train our topic model.
</div>

## Imports


In [6]:
import os
print(os.getcwd())
#!py -m spacy download en_core_web_sm

import spacy
from util.conferences import get_conference_data
from tqdm.notebook import tqdm
import spacy

try:
    nlp
except:
    nlp = spacy.load("nl_core_news_sm")

C:\Users\matth\Documents\dm_project\src\analyses


## Get Data

In [7]:
conferences_data = get_conference_data()

In [14]:
text_by_Rutte = [conference['text'] for conference in conferences_data[0]]
text_by_de_jonge = [conference['text'] for conference in conferences_data[1]]

merged_texts = [text_by_Rutte, text_by_de_jonge]
merged_texts = [item for sublist in merged_texts for item in sublist]

## Tokenize Texts

In [15]:
processed_texts = [text for text in
                   tqdm(nlp.pipe(merged_texts, n_process=1, disable=["ner", "parser"]), total=len(merged_texts))]

  0%|          | 0/58 [00:00<?, ?it/s]

In [10]:
# processed_texts[:10]

In [16]:
# word.pos_ == 'VERB' and not 
tokenized_texts_lem = [[word.lemma_ for word in processed_text if not word.is_stop and not word.is_punct] for
                       processed_text in processed_texts]

## Vectorization

In [17]:
from gensim.corpora import Dictionary

MIN_DF = 1  # minium document frequency
MAX_DF = 0.6  # maximum document frequency

dictionary = Dictionary(tokenized_texts_lem)  # get the vocabulary
dictionary.filter_extremes(no_below=MIN_DF,
                           no_above=MAX_DF)
corpus = [dictionary.doc2bow(text) for text in tokenized_texts_lem]

## Train Model

In [18]:
from gensim.models.wrappers import LdaMallet

os.environ.update({'MALLET_HOME': r'C:/mallet/'})
PATH_TO_MALLET = r'C:/mallet/bin/mallet.bat'

N_TOPICS = 2
N_ITERATIONS = 1000  # usually 1000 will do

lda = LdaMallet(PATH_TO_MALLET,
                corpus=corpus,
                id2word=dictionary,
                num_topics=N_TOPICS,
                optimize_interval=10,
                iterations=N_ITERATIONS)

## Show Topics

In [19]:
for topic in range(N_TOPICS):
    words = lda.show_topic(topic, 20)
    topic_n_words = ' '.join([word[0] for word in words])
    print('Topic {}: {}'.format(str(topic), topic_n_words))

Topic 0: prikken daadwerkelijk maat vaccinatiegraad gezondheidsraad toegangstest levering besmettingsgraad regio 60 bescherming kortom verantwoorden astrazeneca leveren organiseren naam huisarts routekaart opbouwen
Topic 1: uur hugo jonge avondklok pakket lockdown school winkel helaas maximaal corona kind   variant ervoor afstand feit beperking burgemeester discussie


## Visualisation with pyLDAvis

In [22]:
import gensim

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

pyLDAvis.enable_notebook()

lda_conv = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(lda)

gensimvis.prepare(lda_conv, corpus, dictionary)


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.156316  0.0       1        1  57.254423
0     -0.156316  0.0       2        1  42.745577, topic_info=                 Term        Freq       Total Category  logprob  loglift
1546    daadwerkelijk  131.000000  131.000000  Default  30.0000  30.0000
2248  gezondheidsraad  112.000000  112.000000  Default  29.0000  29.0000
351           prikken  140.000000  140.000000  Default  28.0000  28.0000
3618     toegangstest  110.000000  110.000000  Default  27.0000  27.0000
209              hugo  151.000000  151.000000  Default  26.0000  26.0000
...               ...         ...         ...      ...      ...      ...
784           medisch   56.890965   85.270203   Topic2  -6.0164   0.4452
743              juni   60.954605  119.740170   Topic2  -5.9474   0.1747
162              fase   57.906875   97.435100   Topic2  -5.9987   0.3296
912            spelen   52.827325   70.057576   Topic2  -6.0905   0.5676
355       quarantaine   52.827325   72.084665   Topic2  -6.0905   0.5391

[213 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
10        1  0.986637         2021
1988      1  0.986637           28
1483      1  0.022630           60
1483      2  0.973077           60
534       2  0.984339  aanschaffen
...     ...       ...          ...
1043      1  1.005253     zaterdag
1061      1  0.875526        zwaar
1061      2  0.121909        zwaar
1982      1  0.707179             
1982      2  0.292391             

[253 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])